In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import time
from sklearn.ensemble import GradientBoostingClassifier

In [5]:
def prepare_data(df):    
    #Feature engineering
    fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.09] #feature weights
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64('2014-01-01T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    df['hour'] = (d_times.hour+ d_times.minute/60) * fw[2]
    df['weekday'] = d_times.weekday * fw[3]
    df['day'] = (d_times.dayofyear * fw[4]).astype(int)
    df['month'] = d_times.month * fw[5]
    df['year'] = (d_times.year - 2013) * fw[6]
    df.accuracy = df.accuracy.values * fw[7]
    df = df.drop(['time'], axis=1)
    
    return df

In [6]:
df_train = pd.read_csv('../train.csv',
                               usecols=['row_id','x','y','accuracy','time','place_id'])
df_train = df_train[(df_train.x < 0.1) & (df_train.y < 0.04)]

df_valiation = df_train[df_train.time > 786239 * 0.875]
df_train = df_train[df_train.time <= 786239 * 0.875]

df_valiation = prepare_data(df_valiation)
df_train = prepare_data(df_train)

In [7]:
feature_list = ['x','y','hour','weekday','day','month','year', 'accuracy']
clf = GradientBoostingClassifier(n_estimators=20)
clf.fit(df_train[feature_list], df_train['place_id'])
predicted = np.array(clf.predict_proba(df_valiation[features]))

MemoryError: 